In [30]:
## Importing DES Dataset 
#%pip install numpy
import pandas as pd 
import numpy as np

df = pd.read_csv("/Users/matthewrisley/Desktop/Senior Thesis /es_data-v41/es_data-v4_1.csv")

In [31]:
df.head()

,elec_id,country,date,presidential,legislative_type,elecrule,tier1_formula,tier2_formula,tier3_formula,tier4_formula,...,fh_democracy,p5_democracy,vd_polyarchy,dd_democracy,secondround,thirdround,Source,Additional.Sources,Comment,Unnamed: 50
0,L-AAB-1984-4-17,Antigua & Barbuda,"April 17, 1984",0,1.0,1.0,1.0,NaN,NaN,NaN,...,1.0,0.0,0,1,NaN,NaN,Keesing's. Nohlen et al.,NaN,NaN,NaN
1,L-AAB-1989-3-9,Antigua & Barbuda,"March 09, 1989",0,1.0,1.0,1.0,NaN,NaN,NaN,...,1.0,0.0,0,1,NaN,NaN,CIA world factbook Nohlen et al.,NaN,NaN,NaN
2,L-AAB-1994-3-8,Antigua & Barbuda,"March 08, 1994",0,1.0,1.0,1.0,NaN,NaN,NaN,...,0.0,0.0,0,1,NaN,NaN,CIA world factbook http://cdp.binghamton.edu/e...,NaN,NaN,NaN
3,L-AAB-1999-3-9,Antigua & Barbuda,"March 09, 1999",0,1.0,1.0,1.0,NaN,NaN,NaN,...,0.0,0.0,0,1,NaN,NaN,CIA world factbook Adam Carr's Webpage,NaN,NaN,NaN
4,L-AAB-2004-3-23,Antigua & Barbuda,"March 23, 2004",0,1.0,1.0,1.0,NaN,NaN,NaN,...,1.0,0.0,0,1,NaN,NaN,Carr,ElectionGuide.org; Wikipedia; Gallagher,double check Matt's old coding of electoral sy...,NaN


In [32]:
#Keeping only relevant columns
columns_to_keep = ['elec_id', 'country', 'ccode', 'year','presidential','date',
                   'legislative_type','elecrule','tier1_formula','tier2_formula',
                   'tier3_formula','tier4_formula','multi', 'seats', 'tier1_avemag','tier2_avemag', "tier3_avemag", "tier4_avemag"
                   'tier1_districts', 'tier2_districts', 'tier3_districts', 
                   'tier4_districts','enep', 'enep1', 'enpp', 'enpp1', 'dd_democracy',
                    'bmr_democracy', 'fh_democracy', 'p5_democracy', 'vd_polyarchy']
elec_data = df[columns_to_keep]

KeyError: "['tier2_avemag', 'tier3_avemag', 'tier4_avemagtier1_districts'] not in index"

In [ ]:
#Only years after 1980
elec_data = elec_data[elec_data.year >= 1980]

#Dropping presidential elections
elec_data = elec_data[elec_data.presidential == 0]
#Adding a Column calculating the effective electoral threshold. Need to remove any that have values of -99 or -88.

#First going to check which countries don't have full information about their electoral average magnitudes and why
incomplete_district_info = elec_data[(elec_data.tier1_avemag == -99) | (elec_data.tier1_avemag == -88)]

elec_data['tier1_avemag'] = elec_data['tier1_avemag'].replace([-99,-88], np.nan)
elec_data = elec_data.dropna(subset=['tier1_avemag'])

elec_data['tier1_EffElecThrsh'] = 0.75/(elec_data['tier1_avemag']+1)

#Dropping Un-democratic / inconstently democratic countries & creating a list of the dropped countries
dropped_countries = elec_data[(elec_data.vd_polyarchy == 0)|(elec_data.tier1_avemag == -99) | (elec_data.tier1_avemag == -88)]

# Manually removing instances of countries that aren't useful or shouldn't be there,
# (less than 3 elections in a row during the entire time period basically.) 

countries_to_remove = ["Comoros", "Guinea Bissau", "Macedonia", "Maldives", "Madagascar",
                        "Malawi", "Mali", "Montenegro","Nepal", "Nigeria", "Ukraine" ]
countries_to_remove_df = pd.DataFrame(countries_to_remove, columns=["country"])

#Adding them to the dropped countries dataframe
#dropped_countries = dropped_countries[dropped_countries["country"].isin(countries_to_remove)]
dropped_countries = pd.concat([dropped_countries, countries_to_remove_df], ignore_index=True)

elec_data = elec_data[~elec_data["country"].isin(dropped_countries)]

#Dropping non democratic elections as judged by VDEM

elec_data = elec_data[elec_data.vd_polyarchy == 1]
elec_data.tail(50)

#Creating a list of the countries that are still included after all of these drops
unique_countries = elec_data[['country']].drop_duplicates().reset_index(drop=True)

KeyError: 'tier2_avemag'

In [ ]:
elec_data.to_csv('/Users/matthewrisley/Desktop/Senior Thesis /Cleaning Dataframes/SecondCleanedDESDATA.csv', index=False)
dropped_countries.to_csv('/Users/matthewrisley/Desktop/Senior Thesis /Cleaning Dataframes/ListofDroppedcountriesDESDATA.csv', index=False)
unique_countries.to_csv('/Users/matthewrisley/Desktop/Senior Thesis /Cleaning Dataframes/ListofalluniquecountriesDESDATA.csv', index=False)
incomplete_district_info.to_csv('/Users/matthewrisley/Desktop/Senior Thesis /Cleaning Dataframes/Incompletedistrict_info.csv', index=False)